In [1]:
import numpy as np
import open3d as o3d
import copy

## File IO

In [2]:
stl_mesh = o3d.io.read_triangle_mesh("stl_mesh.stl")
obj_mesh = o3d.io.read_triangle_mesh("obj_mesh.obj")

## Visualization

In [3]:
mesh = obj_mesh
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])

In [4]:
o3d.visualization.draw_geometries([mesh, mesh_frame])

In [5]:
scaled_mesh = copy.deepcopy(mesh)
scaled_mesh.scale(1 / np.max(scaled_mesh.get_max_bound() - scaled_mesh.get_min_bound()), center=scaled_mesh.get_center())
o3d.visualization.draw_geometries([scaled_mesh, mesh_frame])

## Normalization

### Center the Mesh

In [6]:
print(mesh.get_center())
centered_vertices = np.array(mesh.vertices) - mesh.get_center()
mesh.vertices = o3d.utility.Vector3dVector(centered_vertices)
print(mesh.get_center())

[ 0.72569102 -0.68593362 10.31785828]
[-7.68295749e-13 -6.04510797e-13 -5.92657940e-12]


### Scale the Points

In [7]:
print(mesh.get_min_bound(), mesh.get_max_bound())
mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())
print(mesh.get_min_bound(), mesh.get_max_bound())

[-33.08519572  -4.25436637 -42.67735916] [31.63381368  2.27343359 75.54260773]
[-0.27986132 -0.03598687 -0.36099959] [0.26758436 0.01923054 0.63900041]


In [8]:
o3d.visualization.draw_geometries([mesh, mesh_frame])

 ## Alignment

### Find Axis with Minimum Moment of Inertia

*reference: https://physics.stackexchange.com/questions/426273/how-to-find-the-axis-with-minimum-moment-of-inertia*

In [9]:
points = np.asarray(mesh.vertices)
points_squared = np.square(points)

In [10]:
# horizontal sum of Y and Z components
I_xx = points_squared[:, 1:].sum(axis=1)
# vertical sum
I_xx = I_xx.sum(axis=0)

# horizontal product of X and Y components
I_xy = points[:, 0] * points[:, 1]
# vertical sum
I_xy = -I_xy.sum(axis=0)

# horizontal product of X and Z components
I_xz = points[:, 0] * points[:, 2]
# vertical sum
I_xz = -I_xz.sum(axis=0)

# horizontal sum of X and Z components
I_yy = points_squared[:, 0] + points_squared[:, 2]
# vertical sum
I_yy = I_yy.sum(axis=0)

# horizontal product of Y and Z components
I_yz = points[:, 1] * points[:, 2]
# vertical sum
I_yz = -I_yz.sum(axis=0)

# horizontal sum of X and Y components
I_zz = points_squared[:, 0] + points_squared[:, 1]
# vertical sum
I_zz = I_zz.sum(axis=0)

In [11]:
inertia_tensor = np.array([[I_xx, I_xy, I_xz], [I_xy, I_yy, I_yz], [I_xz, I_yz, I_zz]])
eigen_values, eigen_vectors = np.linalg.eigh(inertia_tensor)
min_eigen_value_index = eigen_values.argmin()
min_MOI_axis = eigen_vectors[:, min_eigen_value_index]

### Align 2 Unit Vectors

references:
1. https://math.stackexchange.com/questions/180418/calculate-rotation-matrix-to-align-vector-a-to-vector-b-in-3d
2. https://stackoverflow.com/questions/67017134/find-rotation-matrix-to-align-two-vectors

In [12]:
def align_vectors_v1(a, b):
    b = b / np.linalg.norm(b) # normalize a
    a = a / np.linalg.norm(a) # normalize b
    v = np.cross(a, b)
    # s = np.linalg.norm(v)
    c = np.dot(a, b)

    v1, v2, v3 = v
    h = 1 / (1 + c)

    Vmat = np.array([[0, -v3, v2],
                  [v3, 0, -v1],
                  [-v2, v1, 0]])

    R = np.eye(3, dtype=np.float64) + Vmat + (Vmat.dot(Vmat) * h)
    return R

In [13]:
def align_vectors_v2(a, b):
    v = np.cross(a, b)
    s = np.linalg.norm(v)
    c = np.dot(a, b)

    v1, v2, v3 = v
    h = 1 / (1 + c)

    Vmat = np.array([[0, -v3, v2],
                  [v3, 0, -v1],
                  [-v2, v1, 0]])

    R = np.eye(3, dtype=np.float64) + Vmat + (Vmat.dot(Vmat) * h)
    return R

In [14]:
def angle(a, b):
    """Angle between vectors"""
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return np.arccos(a.dot(b))

### Visualize Alignment

In [15]:
axis = np.array([0., 0., 1.])
rotation = align_vectors_v1(min_MOI_axis, axis)

In [16]:
min_MOI_axis = rotation.dot(min_MOI_axis)
vertices_rotated = np.asarray(mesh.vertices).dot(rotation)
mesh.vertices = o3d.utility.Vector3dVector(vertices_rotated)

In [25]:
angle(min_MOI_axis, axis)

0.0

In [17]:
points = [
    [0, 0, 0],
    min_MOI_axis
]

lines = [
    [0, 1]
]

line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines)
)

line_set.scale(2, [0, 0, 0])

LineSet with 1 lines.

In [18]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
o3d.visualization.draw_geometries([mesh, mesh_frame, line_set])

## Voxelization

In [20]:
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size=0.02)
o3d.visualization.draw_geometries([voxel_grid])

In [24]:
np.array(voxel_grid.get_voxels())[:5]

array([Voxel with grid_index: (5, 3, 23), color: (0, 0, 0),
       Voxel with grid_index: (2, 2, 21), color: (0, 0, 0),
       Voxel with grid_index: (15, 1, 25), color: (0, 0, 0),
       Voxel with grid_index: (1, 1, 9), color: (0, 0, 0),
       Voxel with grid_index: (6, 2, 8), color: (0, 0, 0)], dtype=object)